In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [2]:
data_dir = "../data/intermediary-data/"

#todo
Edit below to create 

In [5]:
#  make a lil function to extract the data for top 20 icd codes and attach the new labels/descriptions

def subset_data(df, icd9_data):
    #extract icd9 codes to list
    icd9_codes = icd9_data["icd9_code"].values.tolist()
    
    #subset data based on whether the code is in the desired list
    df_subset = df[df["label"].isin(icd9_codes)].copy()
    
    # get mappping dictionary for code -> triage category
    cat_map = map_codes(icd9_data)
    
    # create a new column with that mapping of ic9_code/label to triage category
    df_subset["triage-category"] = df_subset["label"].map(cat_map)
    
    # data has annoying unnamed column to drop
    try:
        df_subset.drop(columns=["Unnamed: 0"], inplace = True)
    except:
        print("no unnamed col to drop")

    return df_subset
    
    
def map_codes(icd9_data):
    
    '''
    Function to map icd9_code to triage category
    '''
    mapping = {}
    for i,row in icd9_data.iterrows():
        code = row['icd9_code']
        category = row['Triage (post-ICU) Category']

        mapping[code] = category
        
    return mapping


def data_processor(data_dir, icd9_path, modes=["train","valid","test"] ,save_dir = "../data/intermediary-data/triage/"):
    
    # get the icd9 data
    icd9_data = pd.read_csv(f"{data_dir}/{icd9_path}", index_col=None)
    
#     print(icd9_data)
    
    # can create a dataset to return
    dataset = {}
    #run through each provided data mode or set i.e. train/valid/test files   
    for mode in tqdm(modes):
        df = pd.read_csv(f"{data_dir}/top_50_icd9/{mode}.csv", index_col=None)
        df_with_cat = subset_data(df, icd9_data = icd9_data)

        if save_dir is not None:
            print(f"Saving {mode} file at: {save_dir}/{mode}")
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            df_with_cat.to_csv(f"{save_dir}/{mode}.csv", index = None)
            
        dataset[mode] = df_with_cat
        
    return dataset

In [6]:
new_train_data = data_processor(data_dir = f"{data_dir}", icd9_path = f"/triage/top_20_icd9_w_counts_descriptions_grouped.csv", save_dir = None)

 33%|███▎      | 1/3 [00:00<00:01,  1.28it/s]

no unnamed col to drop


100%|██████████| 3/3 [00:01<00:00,  2.38it/s]

no unnamed col to drop


In [7]:
new_train_data

{'train':                                                     text  label  \
 0      : : : Sex: F Service: CARDIOTHORACIC Allergies...   4240   
 1      : : : Sex: F Service: NEONATOLOGY HISTORY: wee...  V3001   
 2      : : : Sex: M Service: CARDIOTHORACIC Allergies...  41041   
 3      : : : Sex: F Service: MEDICINE Allergies: Peni...  51881   
 6      : : : Sex: M Service: ADMISSION DIAGNOSIS: . S...  41401   
 ...                                                  ...    ...   
 14352  : : : Sex: F Service: MEDICINE Allergies: Pati...   5849   
 14353  : : : Sex: F Service: MEDICINE Allergies: Peni...    486   
 14354  Unit No: : : : Sex: F Service: Neonatology was...  V3001   
 14357  : : Service: CARDIOTHORACIC Allergies: Penicil...  41071   
 14358  : : : Sex: M Service: Neonatology HISTORY OF P...  V3001   
 
       triage-category  
 0          Cardiology  
 1          Obstetrics  
 2          Cardiology  
 3         Respiratory  
 6          Cardiology  
 ...               ... 